In [1]:
# curl -X GET -H "Authorization: Bearer <BEARER TOKEN>" "https://api.twitter.com/2/tweets/20"
# bash command to hydrate an example tweet
# get your bearer token (and the keys) here -> https://developer.twitter.com/en/portal/dashboard

In [2]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials['CONSUMER_KEY'] = "xxx"  # key names not consistent ->  API KEY = CONSUMER_KEY
credentials['CONSUMER_SECRET'] = "xxx" # API SECRET KEY = CONSUMER_SECRET
credentials['ACCESS_TOKEN'] = "xxx"   # not needed for now?
credentials['ACCESS_SECRET'] = "xxx" # not needed for now?
credentials['BEARER_TOKEN'] = 'xxx' # seee celll above

 

In [3]:
#!pip install twython
#!pip install geopy

In [3]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file

with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(credentials['CONSUMER_KEY'], credentials['CONSUMER_SECRET'],credentials['ACCESS_TOKEN'],credentials['ACCESS_SECRET'],credentials['BEARER_TOKEN'])


In [4]:
## show json for a specific ID
from pprint import pprint
tweet= python_tweets.show_status(id="1245219745311404032")
#pprint(tweet)

# path to location. but path varies, so it may fail for other IDS
# -> it fails for every tweet that is not this tweet (see evaluation in next ceel)
# -> TODO better generalization / method
#pprint (tweet["extended_entities"]["media"][0]["additional_media_info"]["source_user"]['location'])
#print(tweet["place"]["country"])
#print(tweet["created_at"].split()[1])
#print(tweet["created_at"].split()[2])


In [5]:
# !pip install tqdm

# request limit: 900 requestis/15 mins.
# so if you make 1 request per second, everything will be fine. (15*60 secs = 900secs)
# but it seems to be a rolling window, so you don't have to wait for a full reset, when limit is reached. 
# adjust sleeping time as required. 
# DONE dynamische Berechnung der time-limits um ensprechend anfragefrequenz automatisch regeln

import time

def calc_delay_to_not_run_into_rate_limit():
    # good if you make requests over long time (>15 mins). if not, shorter delay times then returned here would be ok
    # calculation based on the 'python_tweets.show_status(id)'-method. if we use a batch-request-method other time limits apply.
    python_tweets.get_application_rate_limit_status(resources="statuses")
    resource = python_tweets.get_application_rate_limit_status() # if you get a permession denied error here, you have to change your "project settings" here (developer.twitter.com), to "Read, Write, and Direct Messages" and regenerate all keys. TODO add to readme
    resource = resource["resources"]["statuses"]["/statuses/show/:id"] 
    # -> {'limit': 900, 'remaining': 737, 'reset': 1607445919}
    should_sleep = (resource["reset"] - time.time()) / resource["remaining"]
    return should_sleep # seconds to sleep before next request


In [10]:
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd

count = 0
directory = "data/GeoCOV19TweetsDataset"

# For each file/day
for entry in os.scandir(directory):
    count+=1
    
    # Load csv file containing the tweet ID's
    tweets_csv =  pd.read_csv(entry.path)
    tweets_csv.columns = ["ID","Sentiment"] 
    del tweets_csv["Sentiment"] # del sentiment column

    # Inialize panda dataframe for the tweets
    # There will probably be one DF for each day
    Twitter_Tweets = pd.DataFrame(
    columns=['ID', 'COUNTRY', 'DAY', 'MONTH', 'TEXT_RAW'])
    # Only countries from the following list are considered 
    vonInteresse=["Vereinigte Staaten", "Republik Indien", "Vereinigtes Königreich"]

    # For each tweet in the file
    tweet_not_found=0
    for i in tqdm(range(len(tweets_csv))):
        # Delay the tweet extraction since otherwise the rate limit exceeds 
        if i%50 == 0:
            delay = calc_delay_to_not_run_into_rate_limit()
        time.sleep(delay)           

        ID = int(tweets_csv['ID'].values[i])
        try:
            tweet= python_tweets.show_status(id=ID)
        except Exception as e: 
            tweet_not_found += 1
            continue

        # If all values exist add the tweets ID, Country, Day, Month and raw text to the DF
        try:
            if str(tweet["place"]["country"]) in vonInteresse:
                data = {'ID': ID, 
                'COUNTRY': tweet["place"]["country"], 
                'DAY': tweet["created_at"].split()[2],
                'MONTH': tweet["created_at"].split()[1],
                'TEXT_RAW': tweet["text"]} 
                Twitter_Tweets = Twitter_Tweets.append(data, ignore_index=True)
                print(tweet["text"])
                print("")
        except: 
            continue

    # Save DF to csv file
    Twitter_Tweets.to_csv ("data/Tweets/" + os.path.basename(entry.path), index = False, header=True)
    print("")
    print("File " + str(count) + " contains: " + str(len(Twitter_Tweets)) + " Files") 
    print("")        

  0%|▏                                                                                 | 1/533 [00:01<11:00,  1.24s/it]


KeyboardInterrupt: 